In [61]:
import pandas as pd
import numpy as np

import optuna


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adagrad, Adadelta, Nadam

from sklearn import metrics

In [178]:
data = pd.read_csv('Salary_Data.csv')

data

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0
...,...,...,...,...,...,...
6699,49.0,Female,PhD,Director of Marketing,20.0,200000.0
6700,32.0,Male,High School,Sales Associate,3.0,50000.0
6701,30.0,Female,Bachelor's Degree,Financial Manager,4.0,55000.0
6702,46.0,Male,Master's Degree,Marketing Manager,14.0,140000.0


In [63]:
data.describe(include='all')

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
count,6702.000000,6702,6701,6702,6701.000000,6699.000000
unique,NaN,3,7,193,NaN,NaN
top,NaN,Male,Bachelor's Degree,Software Engineer,NaN,NaN
freq,NaN,3674,2267,518,NaN,NaN
mean,33.620859,NaN,NaN,NaN,8.094687,115326.964771
std,7.614633,NaN,NaN,NaN,6.059003,52786.183911
min,21.000000,NaN,NaN,NaN,0.000000,350.000000
25%,28.000000,NaN,NaN,NaN,3.000000,70000.000000
50%,32.000000,NaN,NaN,NaN,7.000000,115000.000000
75%,38.000000,NaN,NaN,NaN,12.000000,160000.000000


In [64]:
data.isnull().sum()

Age                    2
Gender                 2
Education Level        3
Job Title              2
Years of Experience    3
Salary                 5
dtype: int64

In [65]:
data = data.dropna()
data.isnull().sum()

Age                    0
Gender                 0
Education Level        0
Job Title              0
Years of Experience    0
Salary                 0
dtype: int64

In [66]:
data['Education Level'].value_counts()

Education Level
Bachelor's Degree    2265
Master's Degree      1572
PhD                  1368
Bachelor's            756
High School           448
Master's              288
phD                     1
Name: count, dtype: int64

In [67]:

data['Education Level'] = data['Education Level'].replace({
    "Bachelor's": "Bachelor's Degree",
    "Master's": "Master's Degree",
    "phD": "PhD"
})


C:\Users\user\AppData\Local\Temp\ipykernel_5420\2002006214.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Education Level'] = data['Education Level'].replace({


In [68]:
data['Education Level'].value_counts()

Education Level
Bachelor's Degree    3021
Master's Degree      1860
PhD                  1369
High School           448
Name: count, dtype: int64

In [69]:
statistics = ['mean', 'sum']

In [70]:
for i in statistics:
    result = data.groupby(['Job Title']).agg({'Years of Experience': [i]})
    result.columns = [f'Years_of_Experience_{i}_by_Job_Title']
    result.reset_index(inplace=True)
    
    data = pd.merge(data, result, on='Job Title', how='left')
    
data

,Age,Gender,Education Level,Job Title,Years of Experience,Salary,Years_of_Experience_mean_by_Job_Title,Years_of_Experience_sum_by_Job_Title
0,32.0,Male,Bachelor's Degree,Software Engineer,5.0,90000.0,4.449807,2305.0
1,28.0,Female,Master's Degree,Data Analyst,3.0,65000.0,4.969697,1804.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0,17.500000,35.0
3,36.0,Female,Bachelor's Degree,Sales Associate,7.0,60000.0,1.428571,100.0
4,52.0,Male,Master's Degree,Director,20.0,200000.0,20.000000,20.0
...,...,...,...,...,...,...,...,...
6693,49.0,Female,PhD,Director of Marketing,20.0,200000.0,12.068182,1062.0
6694,32.0,Male,High School,Sales Associate,3.0,50000.0,1.428571,100.0
6695,30.0,Female,Bachelor's Degree,Financial Manager,4.0,55000.0,10.694030,1433.0
6696,46.0,Male,Master's Degree,Marketing Manager,14.0,140000.0,9.454902,2411.0


In [71]:
for i in data.columns[-2:]:
    print(i)
    print(data[i].value_counts())
    

Years_of_Experience_mean_by_Job_Title
Years_of_Experience_mean_by_Job_Title
4.449807     518
11.143488    453
16.058511    376
4.969697     363
14.355346    318
            ... 
13.000000      2
5.000000       2
17.500000      2
25.000000      1
24.000000      1
Name: count, Length: 108, dtype: int64
Years_of_Experience_sum_by_Job_Title
Years_of_Experience_sum_by_Job_Title
2305.0    518
5048.0    453
6038.0    376
1804.0    363
4565.0    318
         ... 
35.0        2
21.0        2
4.0         2
39.0        2
16.0        1
Name: count, Length: 106, dtype: int64


In [72]:
data = data.drop('Job Title', axis=1)


In [73]:
data

,Age,Gender,Education Level,Years of Experience,Salary,Years_of_Experience_mean_by_Job_Title,Years_of_Experience_sum_by_Job_Title
0,32.0,Male,Bachelor's Degree,5.0,90000.0,4.449807,2305.0
1,28.0,Female,Master's Degree,3.0,65000.0,4.969697,1804.0
2,45.0,Male,PhD,15.0,150000.0,17.500000,35.0
3,36.0,Female,Bachelor's Degree,7.0,60000.0,1.428571,100.0
4,52.0,Male,Master's Degree,20.0,200000.0,20.000000,20.0
...,...,...,...,...,...,...,...
6693,49.0,Female,PhD,20.0,200000.0,12.068182,1062.0
6694,32.0,Male,High School,3.0,50000.0,1.428571,100.0
6695,30.0,Female,Bachelor's Degree,4.0,55000.0,10.694030,1433.0
6696,46.0,Male,Master's Degree,14.0,140000.0,9.454902,2411.0


In [74]:
data = pd.get_dummies(data, drop_first=True,dtype=int)

data

,Age,Years of Experience,Salary,Years_of_Experience_mean_by_Job_Title,Years_of_Experience_sum_by_Job_Title,Gender_Male,Gender_Other,Education Level_High School,Education Level_Master's Degree,Education Level_PhD
0,32.0,5.0,90000.0,4.449807,2305.0,1,0,0,0,0
1,28.0,3.0,65000.0,4.969697,1804.0,0,0,0,1,0
2,45.0,15.0,150000.0,17.500000,35.0,1,0,0,0,1
3,36.0,7.0,60000.0,1.428571,100.0,0,0,0,0,0
4,52.0,20.0,200000.0,20.000000,20.0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
6693,49.0,20.0,200000.0,12.068182,1062.0,0,0,0,0,1
6694,32.0,3.0,50000.0,1.428571,100.0,1,0,1,0,0
6695,30.0,4.0,55000.0,10.694030,1433.0,0,0,0,0,0
6696,46.0,14.0,140000.0,9.454902,2411.0,1,0,0,1,0


In [75]:
targets = data['Salary']

inputs = data.drop(['Salary'],axis=1)

In [76]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(inputs)

scaled = scaler.transform(inputs)

inputs_scaled = pd.DataFrame(scaled, columns=inputs.columns)

inputs_scaled

,Age,Years of Experience,Years_of_Experience_mean_by_Job_Title,Years_of_Experience_sum_by_Job_Title,Gender_Male,Gender_Other,Education Level_High School,Education Level_Master's Degree,Education Level_PhD
0,-0.213129,-0.510769,-0.811518,0.204385,0.908059,-0.045766,-0.267731,-0.620045,-0.506849
1,-0.738393,-0.840811,-0.695782,-0.078075,-1.101250,-0.045766,-0.267731,1.612785,-0.506849
2,1.493980,1.139440,2.093664,-1.075422,0.908059,-0.045766,-0.267731,-0.620045,1.972973
3,0.312135,-0.180727,-1.484093,-1.038776,-1.101250,-0.045766,-0.267731,-0.620045,-0.506849
4,2.413192,1.964544,2.650204,-1.083879,0.908059,-0.045766,-0.267731,1.612785,-0.506849
...,...,...,...,...,...,...,...,...,...
6693,2.019244,1.964544,0.884454,-0.496408,-1.101250,-0.045766,-0.267731,-0.620045,1.972973
6694,-0.213129,-0.840811,-1.484093,-1.038776,0.908059,-0.045766,3.735089,-0.620045,-0.506849
6695,-0.475761,-0.675790,0.578546,-0.287242,-1.101250,-0.045766,-0.267731,-0.620045,-0.506849
6696,1.625296,0.974419,0.302696,0.264147,0.908059,-0.045766,-0.267731,1.612785,-0.506849


In [77]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs_scaled, targets, test_size=0.2, random_state=42)

In [78]:
def create_model(trial):
    # Building artificial neural network
    model = Sequential()

     # we add 2 hidden layers and 1 output layer
    model.add(Dense(units=trial.suggest_int('units_layer1', 6, 32), activation='relu'))
    model.add(Dense(units=trial.suggest_int('units_layer2', 6, 32), activation='relu'))
    model.add(Dense(units=1, activation='relu'))

    # Suggest hyperparameters for the optimizer
    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'sgd', 'rmsprop', 'adagrad'])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    
    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    elif optimizer_name == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer_name == 'adagrad':
        optimizer = Adagrad(learning_rate=learning_rate)
    
    model.compile(optimizer=optimizer, loss='mean_absolute_error', metrics=['mean_absolute_error'])

    
    return model

In [79]:
from sklearn.metrics import mean_absolute_error, r2_score

def optimal(trial):
    
    # Suggest the number of epochs and batch size
    epochs = trial.suggest_int('epochs', 10, 50)
    batch_size = trial.suggest_int('batch_size', 16, 64)
    
    model = create_model(trial)
    
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
    
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)

    
    return r2

study = optuna.create_study(direction='maximize')
study.optimize(optimal, n_trials=10)

print(f"Best trial: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")

[I 2025-09-27 14:03:16,047] A new study created in memory with name: no-name-7a4f3fee-aef5-49b2-868e-dfbb56868ee2


Epoch 1/19


C:\Users\user\AppData\Local\Temp\ipykernel_5420\905111547.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 115484.6562 - mean_absolute_error: 115484.6562
Epoch 2/19
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 115482.7109 - mean_absolute_error: 115482.7109
Epoch 3/19
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 115475.4688 - mean_absolute_error: 115475.4688
Epoch 4/19
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 115338.6797 - mean_absolute_error: 115338.6797
Epoch 5/19
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 48438.6797 - mean_absolute_error: 48438.6797 
Epoch 6/19
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 21280.2715 - mean_absolute_error: 21280.2715
Epoch 7/19
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 21802.8359 - mean_absolute_error: 21802.8359
Epoch 8/19
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 21363.4805 - mean_absolute_error: 21363.4805
Epoch 9/19
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 21479.1973 - mean_absolute_error: 21479.1953
Epoch 10/19
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 21746.7578 - mea

[I 2025-09-27 14:03:27,980] Trial 0 finished with value: 0.4563982815604577 and parameters: {'epochs': 19, 'batch_size': 57, 'units_layer1': 12, 'units_layer2': 18, 'optimizer': 'sgd', 'learning_rate': 0.003201392317295326}. Best is trial 0 with value: 0.4563982815604577.


...

Epoch 1/47


C:\Users\user\AppData\Local\Temp\ipykernel_5420\905111547.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


84/84 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 115483.1719 - mean_absolute_error: 115483.1719
Epoch 2/47
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 115477.9531 - mean_absolute_error: 115477.9531
Epoch 3/47
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 115469.2266 - mean_absolute_error: 115469.2266
Epoch 4/47
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 115455.7734 - mean_absolute_error: 115455.7734
Epoch 5/47
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 115436.4844 - mean_absolute_error: 115436.4844
Epoch 6/47
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 115410.4531 - mean_absolute_error: 115410.4531
Epoch 7/47
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 115376.4844 - mean_absolute_error: 115376.4844
Epoch 8/47
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 115333.7109 - mean_absolute_error: 115333.7109
Epoch 9/47
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 115281.1562 - mean_absolute_error: 115281.1562
Epoch 10/47
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 115217.

[I 2025-09-27 14:06:48,936] Trial 9 finished with value: -3.275657353971801 and parameters: {'epochs': 47, 'batch_size': 64, 'units_layer1': 17, 'units_layer2': 7, 'optimizer': 'rmsprop', 'learning_rate': 0.0010428634918417084}. Best is trial 0 with value: 0.4563982815604577.


Best trial: 0.4563982815604577
Best hyperparameters: {'epochs': 19, 'batch_size': 57, 'units_layer1': 12, 'units_layer2': 18, 'optimizer': 'sgd', 'learning_rate': 0.003201392317295326}


In [80]:
best_params = study.best_trial.params

best_params

{'epochs': 19,
 'batch_size': 57,
 'units_layer1': 12,
 'units_layer2': 18,
 'optimizer': 'sgd',
 'learning_rate': 0.003201392317295326}

In [81]:
# Train the final model with the best hyperparameters

best_model = Sequential()
best_model.add(Dense(units=best_params['units_layer1'], activation='relu'))
best_model.add(Dense(units=best_params['units_layer2'], activation='relu'))
best_model.add(Dense(1, activation='relu'))

In [82]:
if best_params['optimizer'] == 'adam':
    best_optimizer = Adam(learning_rate=best_params['learning_rate'])
elif best_params['optimizer'] == 'sgd':
    best_optimizer = SGD(learning_rate=best_params['learning_rate'])
elif best_params['optimizer'] == 'rmsprop':
    best_optimizer = RMSprop(learning_rate=best_params['learning_rate'])
elif best_params['optimizer'] == 'adagrad':
    best_optimizer = Adagrad(learning_rate=best_params['learning_rate'])


In [83]:
best_model.compile(optimizer=best_optimizer, loss='mean_absolute_error', metrics=['mean_absolute_error'])

In [84]:
def evaluate(model, X_train, y_train, X_test, y_test):
    
    model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'])
    
    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Metrics for training set
    r2_train = r2_score(y_train, y_train_pred)
    mae_train = mean_absolute_error(y_train, y_train_pred)
    
    # Metrics for test set
    r2_test = r2_score(y_test, y_test_pred)
    mae_test = mean_absolute_error(y_test, y_test_pred)
    
    # Results DataFrame
    results = pd.DataFrame({
        'Metric': ['Train', 'Test'],
        'R2': [r2_train, r2_test],
        'MAE': [mae_train, mae_test]
    })
    
    return results



In [85]:
evaluate(best_model, X_train, y_train, X_test, y_test)

Epoch 1/19
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 115484.5859 - mean_absolute_error: 115484.5859
Epoch 2/19
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 115482.6094 - mean_absolute_error: 115482.6094
Epoch 3/19
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 115475.8516 - mean_absolute_error: 115475.8516
Epoch 4/19
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 115376.9766 - mean_absolute_error: 115376.9766
Epoch 5/19
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 54944.8242 - mean_absolute_error: 54944.8242
Epoch 6/19
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 22114.3398 - mean_absolute_error: 22114.3398
Epoch 7/19
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 22218.6133 - mean_absolute_error: 22218.6133
Epoch 8/19
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 20985.3281 - mean_absolute_error: 20985.3281
Epoch 9/19
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 21458.5195 - mean_absolute_error: 21458.5195
Epoch 10/19
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 21139.

,Metric,R2,MAE
0,Train,0.744311,20676.368576
1,Test,0.755755,20561.493950


In [179]:
data

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0
...,...,...,...,...,...,...
6699,49.0,Female,PhD,Director of Marketing,20.0,200000.0
6700,32.0,Male,High School,Sales Associate,3.0,50000.0
6701,30.0,Female,Bachelor's Degree,Financial Manager,4.0,55000.0
6702,46.0,Male,Master's Degree,Marketing Manager,14.0,140000.0


In [181]:


employee_deploy = pd.DataFrame({
    'Age': [32, 28, np.nan, 36, 52, 49, 32, 30, 41, 37,
            29, 40, 34, 50, 26, 31, 38, 44, np.nan, 33],
    'Gender': ['Male', 'Female', 'Other', 'Female', 'Male',
               'Female', 'Male', 'Female', 'Male', 'Female',
               'Male', 'Other', 'Male', 'Female', 'Male',
               'Female', 'Male', 'Female', 'Other', 'Female'],
    'Education Level': ["Bachelor's", "Master's", "PhD", "Bachelor's", "Master's Degree",
                        np.nan, "High School", "phD", "Bachelor's Degree", "Master's",
                        "Bachelor's", "Master's", "PhD", "phD", "Master's",
                        "Bachelor's Degree", "High School", "Master's", np.nan, "Bachelor's"],
    'Job Title': ['Software Engineer', 'Data Analyst', 'Senior Manager', 'Sales Associate', 'Director',
                  'Director of Marketing', 'Sales Associate', np.nan, 'Financial Manager', 'Project Manager',
                  'Software Engineer', 'Data Analyst', np.nan, 'Sales Associate', 'Director',
                  'Director of Marketing', 'Financial Analyst', 'Financial Manager', 'Project Manager', 'Software Engineer'],
    'Years of Experience': [5, 3, 15, 7, 20, 20, 3, 6, 8, 10,
                            4, 5, 12, 8, 2, 7, 9, 14, 6, 3]
})

employee_deploy


,Age,Gender,Education Level,Job Title,Years of Experience
0,32.0,Male,Bachelor's,Software Engineer,5
1,28.0,Female,Master's,Data Analyst,3
2,NaN,Other,PhD,Senior Manager,15
3,36.0,Female,Bachelor's,Sales Associate,7
4,52.0,Male,Master's Degree,Director,20
5,49.0,Female,NaN,Director of Marketing,20
6,32.0,Male,High School,Sales Associate,3
7,30.0,Female,phD,NaN,6
8,41.0,Male,Bachelor's Degree,Financial Manager,8
9,37.0,Female,Master's,Project Manager,10


In [182]:
employee_deploy.describe(include='all')

,Age,Gender,Education Level,Job Title,Years of Experience
count,18.000000,20,18,18,20.000000
unique,NaN,3,7,9,NaN
top,NaN,Female,Master's,Software Engineer,NaN
freq,NaN,9,5,3,NaN
mean,36.777778,NaN,NaN,NaN,8.350000
std,7.818196,NaN,NaN,NaN,5.363375
min,26.000000,NaN,NaN,NaN,2.000000
25%,31.250000,NaN,NaN,NaN,4.750000
50%,35.000000,NaN,NaN,NaN,7.000000
75%,40.750000,NaN,NaN,NaN,10.500000


In [183]:
employee_deploy.isnull().sum()

Age                    2
Gender                 0
Education Level        2
Job Title              2
Years of Experience    0
dtype: int64

In [184]:
employee_deploy = employee_deploy.dropna()
employee_deploy.isnull().sum()

Age                    0
Gender                 0
Education Level        0
Job Title              0
Years of Experience    0
dtype: int64

In [185]:
employee_deploy['Education Level'].value_counts()

Education Level
Master's             5
Bachelor's           4
High School          2
Bachelor's Degree    2
Master's Degree      1
phD                  1
Name: count, dtype: int64

In [186]:

employee_deploy['Education Level'] = employee_deploy['Education Level'].replace({
    "Bachelor's": "Bachelor's Degree",
    "Master's": "Master's Degree",
    "phD": "PhD"
})

C:\Users\user\AppData\Local\Temp\ipykernel_5420\3551556722.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  employee_deploy['Education Level'] = employee_deploy['Education Level'].replace({


In [187]:
employee_deploy['Education Level'].value_counts()

Education Level
Bachelor's Degree    6
Master's Degree      6
High School          2
PhD                  1
Name: count, dtype: int64

In [188]:
statistics = ['mean', 'sum']

In [189]:
for i in statistics:
    result = employee_deploy.groupby(['Job Title']).agg({'Years of Experience': [i]})
    result.columns = [f'Years_of_Experience_{i}_by_Job_Title']
    result.reset_index(inplace=True)
    
    employee_deploy = pd.merge(employee_deploy, result, on='Job Title', how='left')
    
employee_deploy

,Age,Gender,Education Level,Job Title,Years of Experience,Years_of_Experience_mean_by_Job_Title,Years_of_Experience_sum_by_Job_Title
0,32.0,Male,Bachelor's Degree,Software Engineer,5,4.0,12
1,28.0,Female,Master's Degree,Data Analyst,3,4.0,8
2,36.0,Female,Bachelor's Degree,Sales Associate,7,6.0,18
3,52.0,Male,Master's Degree,Director,20,11.0,22
4,32.0,Male,High School,Sales Associate,3,6.0,18
5,41.0,Male,Bachelor's Degree,Financial Manager,8,11.0,22
6,37.0,Female,Master's Degree,Project Manager,10,10.0,10
7,29.0,Male,Bachelor's Degree,Software Engineer,4,4.0,12
8,40.0,Other,Master's Degree,Data Analyst,5,4.0,8
9,50.0,Female,PhD,Sales Associate,8,6.0,18


In [190]:
for i in employee_deploy.columns[-2:]:
    print(i)
    print(employee_deploy[i].value_counts())
    

Years_of_Experience_mean_by_Job_Title
Years_of_Experience_mean_by_Job_Title
4.0     5
11.0    4
6.0     3
10.0    1
7.0     1
9.0     1
Name: count, dtype: int64
Years_of_Experience_sum_by_Job_Title
Years_of_Experience_sum_by_Job_Title
22    4
12    3
18    3
8     2
10    1
7     1
9     1
Name: count, dtype: int64


In [191]:
employee_deploy = employee_deploy.drop('Job Title', axis=1)


In [192]:
employee_deploy1 = pd.get_dummies(employee_deploy,dtype=int)

employee_deploy1

,Age,Years of Experience,Years_of_Experience_mean_by_Job_Title,Years_of_Experience_sum_by_Job_Title,Gender_Female,Gender_Male,Gender_Other,Education Level_Bachelor's Degree,Education Level_High School,Education Level_Master's Degree,Education Level_PhD
0,32.0,5,4.0,12,0,1,0,1,0,0,0
1,28.0,3,4.0,8,1,0,0,0,0,1,0
2,36.0,7,6.0,18,1,0,0,1,0,0,0
3,52.0,20,11.0,22,0,1,0,0,0,1,0
4,32.0,3,6.0,18,0,1,0,0,1,0,0
5,41.0,8,11.0,22,0,1,0,1,0,0,0
6,37.0,10,10.0,10,1,0,0,0,0,1,0
7,29.0,4,4.0,12,0,1,0,1,0,0,0
8,40.0,5,4.0,8,0,0,1,0,0,1,0
9,50.0,8,6.0,18,1,0,0,0,0,0,1


In [193]:
scaler.fit(employee_deploy1)

scaled = scaler.transform(employee_deploy1)

df_test_scaled = pd.DataFrame(scaled, columns=employee_deploy1.columns)

df_test_scaled

,Age,Years of Experience,Years_of_Experience_mean_by_Job_Title,Years_of_Experience_sum_by_Job_Title,Gender_Female,Gender_Male,Gender_Other,Education Level_Bachelor's Degree,Education Level_High School,Education Level_Master's Degree,Education Level_PhD
0,-0.616317,-0.474538,-1.111183,-0.475719,-0.935414,1.069045,-0.267261,1.224745,-0.392232,-0.816497,-0.267261
1,-1.152245,-0.905936,-1.111183,-1.189297,1.069045,-0.935414,-0.267261,-0.816497,-0.392232,1.224745,-0.267261
2,-0.080389,-0.043140,-0.416693,0.594648,1.069045,-0.935414,-0.267261,1.224745,-0.392232,-0.816497,-0.267261
3,2.063323,2.760947,1.319529,1.308227,-0.935414,1.069045,-0.267261,-0.816497,-0.392232,1.224745,-0.267261
4,-0.616317,-0.905936,-0.416693,0.594648,-0.935414,1.069045,-0.267261,-0.816497,2.549510,-0.816497,-0.267261
5,0.589521,0.172559,1.319529,1.308227,-0.935414,1.069045,-0.267261,1.224745,-0.392232,-0.816497,-0.267261
6,0.053593,0.603957,0.972285,-0.832508,1.069045,-0.935414,-0.267261,-0.816497,-0.392232,1.224745,-0.267261
7,-1.018263,-0.690237,-1.111183,-0.475719,-0.935414,1.069045,-0.267261,1.224745,-0.392232,-0.816497,-0.267261
8,0.455539,-0.474538,-1.111183,-1.189297,-0.935414,-0.935414,3.741657,-0.816497,-0.392232,1.224745,-0.267261
9,1.795359,0.172559,-0.416693,0.594648,1.069045,-0.935414,-0.267261,-0.816497,-0.392232,-0.816497,3.741657


In [194]:
X_train.columns

Index(['Age', 'Years of Experience', 'Years_of_Experience_mean_by_Job_Title',
       'Years_of_Experience_sum_by_Job_Title', 'Gender_Male', 'Gender_Other',
       'Education Level_High School', 'Education Level_Master's Degree',
       'Education Level_PhD'],
      dtype='object')

In [195]:

df_test_scaled = df_test_scaled[['Age', 'Years of Experience', 'Years_of_Experience_mean_by_Job_Title',
       'Years_of_Experience_sum_by_Job_Title', 'Gender_Male', 'Gender_Other',
       'Education Level_High School', "Education Level_Master's Degree",
       'Education Level_PhD']]


In [196]:
employee_deploy['predicted_Salary'] = best_model.predict(df_test_scaled)


employee_deploy

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step


,Age,Gender,Education Level,Years of Experience,Years_of_Experience_mean_by_Job_Title,Years_of_Experience_sum_by_Job_Title,predicted_Salary
0,32.0,Male,Bachelor's Degree,5,4.0,12,115800.968750
1,28.0,Female,Master's Degree,3,4.0,8,63198.457031
2,36.0,Female,Bachelor's Degree,7,6.0,18,171505.531250
3,52.0,Male,Master's Degree,20,11.0,22,232691.937500
4,32.0,Male,High School,3,6.0,18,66122.656250
5,41.0,Male,Bachelor's Degree,8,11.0,22,151341.937500
6,37.0,Female,Master's Degree,10,10.0,10,152725.671875
7,29.0,Male,Bachelor's Degree,4,4.0,12,110948.078125
8,40.0,Other,Master's Degree,5,4.0,8,29200.626953
9,50.0,Female,PhD,8,6.0,18,130413.093750
